In [1]:
import rasterio as rio
import numpy as np

In [2]:
# Read
with rio.open('/home/data/ocean-colour/fixed-pattern-noise/detector-footprint-B2_10m_4326.tif', 'r') as src:
    d_footprint = src.read(1)
    no_data_foot = src.nodata
    
with rio.open('/home/data/ocean-colour/fixed-pattern-noise/chla_linear_60m_4326_landsea_clean.tif', 'r') as src:
    chla = src.read(1)
    no_data_chla = src.nodata
    profile = src.profile

In [3]:
N_footprints = np.unique(d_footprint)
N_footprints = N_footprints[N_footprints != no_data_foot]
N_footprints

array([3, 4, 5, 6, 7, 8], dtype=uint8)

In [4]:
valid_chla = list()

for n in N_footprints.tolist():
    footprint_mask = d_footprint == n # get footprint mask for each footprint
    nodata_mask = (d_footprint != no_data_foot) & (chla != no_data_chla)

    valid_data_mask = footprint_mask & nodata_mask
    
    # Get valid values
    valid_chla_foot = chla[valid_data_mask]
    
    valid_chla.append((np.mean(valid_chla_foot), np.median(valid_chla_foot), np.std(valid_chla_foot)))

In [5]:
chla_stats = np.stack(valid_chla)
chla_stats

array([[0.68480736, 0.66438526, 0.14964287],
       [0.3880749 , 0.35794708, 0.20541762],
       [0.25357443, 0.22867428, 0.13483933],
       [0.25415975, 0.2391891 , 0.10029809],
       [0.45058843, 0.425434  , 0.15376009],
       [0.27268258, 0.2632374 , 0.09897326]], dtype=float32)

In [7]:
# set reference chla
detector_n = 2
ref_chla = chla_stats[detector_n, 0]
ref_chla = 0.5

# Compute gain factors per detector footprint
gain_factors =  ref_chla / chla_stats[:, 0]
gain_factors

array([0.73013234, 1.288411  , 1.9718076 , 1.9672667 , 1.10966   ,
       1.8336338 ], dtype=float32)

In [8]:
for i, n in enumerate(N_footprints.tolist()):

    footprint_mask = d_footprint == n # get footprint mask for each footprint
    nodata_chla_mask = chla == no_data_chla
    
    # valid_data_mask = footprint_mask & nodata_mask
    chla[footprint_mask] = chla[footprint_mask] * gain_factors[i]
    
    # Reset No data values
    chla[nodata_chla_mask] = no_data_chla

    # valid_chla.append((np.mean(valid_chla_foot), np.median(valid_chla_foot), np.std(valid_chla_foot)))

In [9]:
# Save output
with rio.open('/home/data/ocean-colour/fixed-pattern-noise/chla_linear_60m_4326_landsea_clean_fixpatternanchor.tif', 'w', **profile) as dst:
    dst.write(chla, 1)